<a href="https://colab.research.google.com/github/lifeisbeautifu1/graduate-work/blob/main/coloring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Neural Network for coloring grey-scale images

In [1]:
# check for gpu
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-25920366-4fb9-4418-064b-a9f3f337df34)


# Alpha version




### Imports

In [5]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [47]:
PATH_TO_DRIVE = "drive/MyDrive/Colab Notebooks/"
IMAGE_SHAPE = 228

image = img_to_array(load_img(PATH_TO_DRIVE + 'woman.jpg'))
image = np.array(image, dtype=float)

In [48]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1,  400, 400, 1)
Y = Y.reshape(1,  400, 400, 2)

In [49]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [50]:
model.compile(optimizer='rmsprop',loss='mse')

In [51]:
model.fit(x=X,
    y=Y,
    batch_size=1,
    verbose=0,
    epochs=1000)

In [52]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

# cur.shape
image_rgb = lab2rgb(cur)
image_rgb *= 255
imsave(PATH_TO_DRIVE + "img_result.png", image_rgb.astype(np.uint8))
imsave(PATH_TO_DRIVE + "img_gray_version.png", rgb2gray(image_rgb).astype(np.uint8))

1/1 [==============================] - 0s 140ms/step - loss: 9.9724e-04
0.000997238210402429
1/1 [==============================] - 0s 103ms/step
